In [ ]:
import pandas as pd
import os
import requests
import json
import time
from notifypy import Notify
import base64
import re

In [ ]:
cwd = os.getcwd()
resources = os.path.join(cwd,'resources')

In [ ]:
with open('params.json', 'r') as file:
    json_string = file.read()
    
params = json.loads(json_string)

autoModifySituations = params['autoModifySituations']
if autoModifySituations:
    verification = input('Aviso: O update automático das situações de pedido está ativado, deseja prosseguir? [s/n]: ')
    
    if verification == 's':
        pass
    else:
        exit()

situationsToGetLogistics = params['situationsToGetLogistics']

situationToNotify = params['situationToNotify']

late_param = params['late_param']
late_timerLoop = params['late_timerLoop']

if not(type(situationsToGetLogistics) == list):
    print('situationsToGetLogistics deve ser uma lista')
    input('Pressione ENTER para sair.')
    exit()

if not(type(situationToNotify) == dict):
    print('situationToNotify deve ser uma lista')
    input('Pressione ENTER para sair.')
    exit()
# print(situationToNotify)

In [ ]:
class BlingDfTreat():
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def autoUpdateSituations(self):
        filtered_series = self.dataframe.logistics.dropna()
        
        if 0 < filtered_series.size < 5:
            for index,value in filtered_series.items():
                no_nfe = bool(re.search('FLEX|ERUPÇÃO',value))
                needs_nfe = ( bool(re.search('ENVIOS',value)) and not bool(re.search('ERUPÇÃO',value)) )

                if no_nfe:
                    BlingRequests.patchBlingSituation(orderId=index, situationId=312879)
                elif needs_nfe:
                    BlingRequests.patchBlingSituation(orderId=index, situationId=312950)
    
    
    def translateLogisticId(self):
        if not all(self.dataframe.logistics.isna()):
            self.dataframe.logistics = self.dataframe.logistics.replace(BlingRequests.getLogisticIdsDict(), regex=True)     

    def addLogisticColumn(self,situationsToGetLogistics:list):   
        situationsToGetLogistics = '|'.join(situationsToGetLogistics)

        mask = self.dataframe.situacao.str.contains(situationsToGetLogistics,regex=True)
        logistic_id_series = self.dataframe.loc[mask].apply(lambda x: BlingRequests.getLogisticIdWithOrderId(x.name), axis=1)
        
        self.dataframe.loc[mask,'logistics'] = logistic_id_series
    
    def selectNameOnContactColumn(self):
        self.dataframe.contato = self.dataframe.contato.apply(lambda x: x['nome'])

    def translateIdOnSituationColumn(self):
        translation = BlingRequests.getSituationIdsDict()
        self.dataframe.situacao = self.dataframe.situacao.replace(translation)

    def selectIdOnSituationColumn(self):
        self.dataframe.situacao = self.dataframe.situacao.apply(lambda x: x['id'])



In [ ]:
class BlingRequests():

    def rawOrdersDf():
        ordersDf = pd.DataFrame(BlingRequests.getOrders()).set_index('id')
        return ordersDf
    
    def getOrders():
        url = 'https://www.bling.com.br/Api/v3/pedidos/vendas'
        response = BlingRequests.get(url)
        data = response['data']
        return data

    def getLogisticIdsDict():
        url = f'https://www.bling.com.br/Api/v3/logisticas'
        response = BlingRequests.get(url)
        logistics = response['data']
        
        logistic_ids_dict = {str(logistic['id']):logistic['descricao'] for logistic in logistics}

        return logistic_ids_dict
    
    def getLogisticIdWithOrderId(orderId): 
        try:
            objectId = BlingRequests.getOrderObjectId(orderId)
            serviceId = BlingRequests.getServiceIdWithObjectId(objectId)
            logisticId = BlingRequests.getLogisticIdWithServiceId(serviceId)
        except IndexError:
            logisticId = 0
        return str(logisticId)

    def getLogisticIdWithServiceId(serviceId):
        url = f'https://www.bling.com.br/Api/v3/logisticas/servicos/{serviceId}'
        response = BlingRequests.get(url)
        logisticId = response['data']['logistica']['id']
        
        return logisticId

    def getServiceIdWithObjectId(objectId):
        url = f'https://www.bling.com.br/Api/v3/logisticas/objetos/{objectId}'
        response = BlingRequests.get(url)
        serviceId = response['data']['servico']['id']

        return serviceId

    def getOrderObjectId(orderId):
        url = f'https://www.bling.com.br/Api/v3/pedidos/vendas/{orderId}'
        response = BlingRequests.get(url)
        objectId = response['data']['transporte']['volumes'][0]['id']
        return objectId

    def patchBlingSituation(orderId:int, situationId:int):
        url = f'https://www.bling.com.br/Api/v3/pedidos/vendas/{orderId}/situacoes/{situationId}'
        requests.patch(url, headers=BlingRequests.header())

    def getSituationIdsDict():
        return {situation['id']:situation['nome'] for situation in BlingRequests.getSituationIds()}
    
    def getSituationIds():
        response = BlingRequests.get(url='https://www.bling.com.br/Api/v3/situacoes/modulos/98310')
        data = response['data']
        return data
    
    def get(url:str):
        return requests.get(url=url, headers=BlingRequests.header()).json()

    def header():
        header = {       
            'Authorization':f'Bearer {BlingAPI.bling_access_token()}'
            ,'accept': 'application/json'
        }
        
        return header

In [ ]:
class BlingAuth():

    def getBlingClient():
        with open('params.json','r') as file:
            params_string = file.read()

        params = json.loads(params_string)
        return params

    def encode_to_base64(string):
        encoded_bytes = base64.b64encode(string.encode('utf-8'))
        encoded_string = encoded_bytes.decode('utf-8')
        return encoded_string

    client_id = getBlingClient()['bling_client_id']
    client_secret = getBlingClient()['bling_client_secret']
    encoded_authorization = encode_to_base64(f'{client_id}:{client_secret}')

    def updateBlingToken():
        if not os.path.exists('bling_tokens.json'):
            BlingAuth.getFirstBlingToken()
        else:
            BlingAuth.refreshTokens()

    def getFirstBlingToken():
        url = 'https://www.bling.com.br/Api/v3/oauth/token'

        state = 'expedicao'
        authorization_url = f'https://www.bling.com.br/Api/v3/oauth/authorize?response_type=code&client_id={BlingAuth.client_id}&state={state}'
        print('Para obter o authorization_code, acesse este site:\n',authorization_url)

        authorization_code = input('authorization_code here: ')

        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
            ,'Accept': '1.0'
            ,'Authorization': f'Basic {BlingAuth.encoded_authorization}'
        }

        body = {
            'grant_type':'authorization_code'
            ,'code':f'{authorization_code}'
        }

        token_response = requests.post(url,headers=headers, data=body).json()
        print(token_response)
        bling_tokens = {k:v for k,v in token_response.items() if k == 'access_token' or k == 'refresh_token'}

        if len(bling_tokens) >= 2:
            with open('bling_tokens.json', 'w') as file:
                json.dump(bling_tokens,file,indent=4)
        else:
            input('Dados de token vazios. Pressione ENTER para continuar...')


    def refreshTokens():
        url = 'https://www.bling.com.br/Api/v3/oauth/token'
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded'
            ,'Accept': '1.0'
            ,'Authorization': f'Basic {BlingAuth.encoded_authorization}'
        }

        data = {
            'grant_type':'refresh_token'
            ,'refresh_token':BlingAPI.bling_refresh_token()
        }

        token_response = requests.post(url,data=data,headers=headers).json()
        bling_tokens = {k:v for k,v in token_response.items() if k == 'access_token' or k == 'refresh_token'}
        
        with open('bling_tokens.json', 'w') as file:
            json.dump(bling_tokens,file,indent=4)

In [ ]:
class BlingAPI:
    def read_bling_token():
        with open('bling_tokens.json', 'r') as file:
            json_string = file.read()

        bling_tokens = json.loads(json_string)

        return bling_tokens
    
    def bling_access_token():
        return BlingAPI.read_bling_token()['access_token']
    
    def bling_refresh_token():
        return BlingAPI.read_bling_token()['refresh_token']

In [ ]:
def getNewOrderNotification(listOfNewOrders : list):
    notify_new = Notify()
    notify_new.application_name = 'Monitor Bling'
    notify_new.title = 'Nova Venda'
    notify_new.message = ', '.join([str(i) for i in listOfNewOrders])

    # notify_new.audio = (os.path.join(resources,'notify_new.wav'))

    notify_new.send(block=False)

def getLateOrderNotification(listOfLateOrders : list):
    notify_old = Notify()
    notify_old.application_name = 'Monitor Bling'
    notify_old.title = 'Vendas em Atraso'
    notify_old.message = ', '.join([str(i) for i in listOfLateOrders])

    # notify_old.audio = (os.path.join(resources,'notify_old.wav'))

    notify_old.send(block=False)

In [ ]:
waitingOrdersDict = {}
minutes_running = 0
late_timer = 0

testloop = True
loop_interval = (60 - 4)

while testloop:
    try:
        bling = BlingDfTreat(BlingRequests.rawOrdersDf())
        bling.selectIdOnSituationColumn()
        bling.translateIdOnSituationColumn()
        bling.selectNameOnContactColumn()
        bling.addLogisticColumn(situationsToGetLogistics)
        bling.translateLogisticId()

    except:
        print('error fetching data from API')
        BlingAuth.updateBlingToken()
        print('test')
        time.sleep(1)
        continue
    
    if autoModifySituations:
        bling.autoUpdateSituations()
        
    waitingOrdersMask = bling.dataframe.situacao.str.contains('|'.join(situationToNotify.keys()))
    waitingOrdersDf = bling.dataframe.loc[waitingOrdersMask]

    if waitingOrdersMask.any():
        for i,v in waitingOrdersDf.iterrows():
            if i not in waitingOrdersDict:
                waitingOrdersDict[i] = {
                    'num':v.numero 
                    ,'situation':v.situacao
                    ,'logistic':v.logistics 
                    ,'count':0
                } 
            else:
                waitingOrdersDict[i]['situation'] = v.situacao
                waitingOrdersDict[i]['logistic'] = v.logistics
                waitingOrdersDict[i]['count'] += 1
        
         # removes attended orders
        for i in list(waitingOrdersDict.keys()):
            if  i not in waitingOrdersDf.index:
                waitingOrdersDict.pop(i)
    else:
        waitingOrdersDict = {}

    os.system('cls')

    for situation in situationToNotify.keys():
        ordersBySituation_dict = {k:v for k,v in waitingOrdersDict.items() if re.search(situation,v['situation'])}

        new = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict if ordersBySituation_dict[i]['count'] == 0]
        waiting = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict\
                if (0 < ordersBySituation_dict[i]['count'] < late_param)]
        late = [ordersBySituation_dict[i]['num'] for i in ordersBySituation_dict if ordersBySituation_dict[i]['count'] >= late_param]

        print(situation.replace('\\',''))
        print('new orders: ', new)
        print('order waiting: ', waiting)
        print(f'orders waiting for {late_param}min or more: ', late,'\n')
        print('#---------------------------------------------------#\n')

    print(f'program running for: {minutes_running} minutes\n')
    minutes_running += 1

    # dict used for notifications
    new = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict if waitingOrdersDict[i]['count'] == 0 and\
            situationToNotify[waitingOrdersDict[i]['situation']\
                .replace('(','\(').replace(')','\)')]\
                    ['notifyNew']]
    
    waiting = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict\
            if (0 < waitingOrdersDict[i]['count'] < late_param)]
    
    late = [waitingOrdersDict[i]['num'] for i in waitingOrdersDict\
            if waitingOrdersDict[i]['count'] >= late_param and\
                situationToNotify[waitingOrdersDict[i]['situation']\
                    .replace('(','\(').replace(')','\)')]\
                        ['notifyOld']]
    
    if new:
        getNewOrderNotification(new)
        late_timer = 0

    elif late and not waiting:
        if late_timer == 0 or late_timer == late_timerLoop:
            late_timer = 1
            getLateOrderNotification(late)
        elif late_timer < late_timerLoop:
            late_timer += 1
    
    # testloop = False
    time.sleep(loop_interval)
